In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model = "meta-llama/Llama-2-7b-hf"
peft_model = "FinGPT/fingpt-mt_llama2-7b_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", load_in_4bit=True, torch_dtype=torch.float16)

model = PeftModel.from_pretrained(model, peft_model, device_map="auto", torch_dtype=torch.float16)

model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import sys
sys.path.append('../../benchmarks/')

from sentiment140 import test_sentiment140

dataset = test_sentiment140(model, tokenizer)



Prompt example:
Instruction: What is the sentiment of this twitter? Please choose an answer from {negative/neutral/positive}.
Input: Reading my kindle2...  Love it... Lee childs is good read.
Answer: 


Total len: 150. Batchsize: 8. Total steps: 19


  0%|          | 0/19 [00:00<?, ?it/s]/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 19/19 [00:35<00:00,  1.85s/it]

Acc: 0.4266666666666667. F1 macro: 0.34259406395112735. F1 micro: 0.42666666666666675. F1 weighted (BloombergGPT): 0.34259406395112735. 


In [4]:
path = '../../results/general/llama2-7b-fingpt.csv'

In [5]:
dataset.to_csv(path)

In [6]:
import pandas as pd

df = pd.read_csv(path, index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.789, Recall: 0.427, F1: 0.343, Accuracy: 0.427
